In [ ]:
import networkx as nx
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from networkx.algorithms import community as cm

## 3.1 Introdução 

Uma das tarefas comum em análise de redes complexas é agrupar os vértices da rede em subconjuntos, chamados de comunidades, de modo que os vértices mais similares fiquem juntos em uma comunidade. 

A ideia central é entender um pouco sobre como os nós tendem a se organizar dentro da rede através da observação dos padrões de ligação que os mesmos possuem.

Problemas de detecção de comunidades são similares a problemas de partição em grafos, mas em geral nestes últimos já há uma quantidade pré-determinada de grupos e dos tamanhos dos mesmos no qual o grafo deve ser particionado, o que não ocorre em geral com os problemas de detecção de comunidades nos quais o tamanho e a quantidade de comunidades a serem detectadas não são previamente conhecidos

## 3.2 CONCOR

É um dos primeiros e mais usados métodos para detectar comunidades. 

Esse método é baseado na convergência de correlações iteradas.

O método **CONCOR** aplicado uma única vez, divide o conjunto de nós em no máximo duas comunidades. 
Se mais comunidades forem necessárias, pode-se aplicar novamente o método **CONCOR** em uma comunidade encontrada anteriormente, dividindo-a em duas.

### Exemplo 1:

Considere a instância do livro 2 da saga Harry Potter. 

As ligações dessa rede representam suporte emocional e, portanto, trata-se de uma rede direcionada com matriz de adjacência não simétrica.

Usamos o algoritmo **CONCOR** para achar uma divisão em duas comunidades.

In [ ]:
# lendo os arquivos
atri = pd.read_csv('../../data/harrypotter/hpattributes.txt', sep='\t') 
ares = pd.read_csv('../../data/harrypotter/hpbook2.txt', sep=' ', header=None)
nome = pd.read_csv('../../data/harrypotter/hpnames.txt', sep='\t')

In [ ]:
# criando o grafo(rede)
gpotter = nx.DiGraph()
n = atri.shape[0]

for k in range(n):
    gpotter.add_node(k,
                     nome = nome['name'][k],
                     ano = atri['schoolyear'][k],
                     gen = atri['gender'][k],
                     casa = atri['house'][k])

for k in range(n):
    for m in range(n):
        if ares.values[k][m] == 1:
            gpotter.add_edge(k,m)

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,10))

nx.draw(gpotter, with_labels=True)

In [ ]:
ldegree0 = []
for i in gpotter.nodes:
    if gpotter.degree[i] == 0:
        ldegree0.append(i)

print(ldegree0)
print(len(ldegree0))

In [ ]:
# gerando a matriz de adjacência esparsa
#A = nx.adjacency_matrix(gpotter)
A = nx.to_scipy_sparse_array(gpotter).toarray()

#print(f"adjacency : \n {A}")
print(A)

Excluímos os nós isolados, pois estes vértices podem ser considerados uma comunidade separada, mas alguns deles correspondem a personagens que não fazem parte do livro.

In [ ]:
# deletando os nodes isolados

isolados = []
for k in range(n):
    if np.sum(A[[k],:]) == 0:
        isolados.append(k)
        gpotter.remove_node(k)

print(len(isolados), 'nos isolados')

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,10))

nx.draw(gpotter, with_labels=True)

In [ ]:
# matriz de adjacencia esparsa
#A = nx.adjacency_matrix(gpotter)
A = nx.to_scipy_sparse_array(gpotter)#.toarray()

# matriz de adjacencia densa
A = np.array(A.todense(),dtype=float)
print(A)

A convergência do algoritmo é detectada usando a função **allclose** do **Numpy**. 
Quando trabalhamos com aritmética de ponto flutante, existe a possibilidade de que os elementos da matriz $C_k$ não fiquem exatamente iguais a +-1. 

As correlações são feitas usando a função **corr_coef** de Numpy. 

O método **adjacency_matrix**(**to_scipy_sparse_array**) devolve a matriz em um formato para matrizes esparsas, assim convertemos a matriz para um formato denso com método **todense**.

**allclose()**: retorna True se dois arrays forem iguais elemento por elemento dentro de uma tolerância.

**ones_like():** retorna um array de uns com o mesmo formato e tipo de um array fornecido.

 **corrcoef():** retorna os coeficientes de correlação produto-momento de Pearson.

In [ ]:
# algoritmo CONCOR
C = A.copy()

while not np.allclose(np.abs(C), np.ones_like(C), atol=1e-10):
    C = np.corrcoef(C)

C = np.round(C)

print(C)

In [ ]:
com1 = [] # comunidade 1
com2 = [] # comunidade 1

nos  = list(gpotter.nodes)

com1.append(nos[0]) # adiciona o node 0

for k in range(1,len(nos)):
    if C[0,k] > 0:
        com1.append(nos[k])
    else:
        com2.append(nos[k])

In [ ]:
print(f"#com1 = {len(com1)}, #com2 = {len(com2)}")

In [ ]:
print('comunidade 1:')
for k in range(len(com1)):
    print(f"{com1[k]}: {gpotter.nodes[com1[k]]['nome']}")

In [ ]:
print('comunidade 2:')
for k in range(len(com2)):
    print(f"{com2[k]}: {gpotter.nodes[com2[k]]['nome']}")

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 100*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 100*np.sin(2*np.pi*k/len(com1))

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')
    
etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size=500, labels=etiquetas,
                 font_size=10, node_color=cores)

plt.box(False)
plt.show()

### Exemplo 2

Os problemas partição de um grafo em comunidades são, comumente, muito complexos. 
Assim existem heurísticas para resolver esse tipo de problema.

A função **heuristic_community** minimiza a razão entre o número de arestas que ligam dois grupos e o produto dos tamanhos dos dois grupos.

O pacote **algorithms.community** do **networkx** contém funções para calcular e medir a estrutura de comunidades em redes. 

Um exemplo é a função **partition_quality**, que retorna a razão entre o número de arestas internas aos grupos e o número total de arestas.

In [ ]:
a,b = cm.partition_quality(gpotter,[com1,com2])
print(a)
print(b)

In [ ]:
def heuristic_community(G):
    mper = np.inf
    for ini in G:
        com1 = set([ini])
        com2 = set(nos)-com1
    
        mgan = -1 # mgan = Delta
        while mgan < 0:
            mgan = np.inf
            cv, pe = cm.partition_quality(G,[com1,com2]) # calculo da razao 
            ra = (1-cv)/(len(com1)*len(com2)) # calcula da razao
            #print(f"cv={cv}, pe={pe}, ra={ra}")
            mno = -1

            for no in com2:
                ncom1 = com1.union(set([no]))
                ncom2 = com2-set([no])
                cv, pe = cm.partition_quality(G,[ncom1,ncom2])
                nra = (1-cv)/(len(ncom1)*len(ncom2)) #nra = delta
                ng = nra-ra

                if ng < mgan:
                    mgan = ng
                    mno  = no

            if mgan < 0:
                com1.add(mno)
                com2.remove(mno)
    
        cv, pe = cm.partition_quality(G,[com1,com2])            
        ra = (1-cv)/(len(com1)*len(com2)) 

        if ra < mper:
            mper = ra
            mcom1 = com1.copy()
            mcom2 = com2.copy()

    return mcom1, mcom2

In [ ]:
mcom1, mcom2 = heuristic_community(gpotter)
com1 = list(mcom1)
com2 = list(mcom2)

In [ ]:
print(f"comunidade 1:")
for k in range(len(com1)):
    print(f"{com1[k]}: {gpotter.nodes[com1[k]]['nome']}")

In [ ]:
print(f"comunidade 2:")
for k in range(len(com2)):
    print(f"{com2[k]}: {gpotter.nodes[com2[k]]['nome']}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,
                 pos=pos,
                 edge_color='lightgray',
                 node_size = 500, 
                 labels=etiquetas,
                 font_size=10, 
                 node_color=cores)

plt.box(False)
plt.show()

## 3.3 Algoritmo de Kernighan-Lin

Um algoritmo de bisseção muito conhecido é o algoritmo de **Kernighan e Lin**. 

Este algoritmo começa com uma partição do grafo e prossegue trocando dois nós, um de cada grupo, de maneira de aprimorar a partição.

No **networkx** está implementado o métodos **kernighan_lin_bisection**, o qual retorna a particão de um grafo em dois blocos usando o algoritmo **Kernighan-Lin**.

Este algoritmo particiona uma rede em dois conjuntos trocando iterativamente pares de nós para reduzir o corte de arestas entre os dois conjuntos.

Os pares são escolhidos de acordo com uma forma modificada do **Kernighan-Lin**, que move os nós individualmente, alternando entre os lados para manter a bisseção equilibrada.

Considere uma rede com um conjunto de nós $V$ e seja $G_1$ , $G_2 \subseteq V$ uma partição de $V$. 

Sejam $I_k$ e $E_k$ os números de arestas associadas ao nó $k$ que são internas e externas ao grupo ao qual $k$ pertence, respectivamente.
Em cada passo, o algoritmo procura encontrar o par $(i,j) \in G_1 \times G_2$ que maximize  $\Delta_{ij} = E_i − I_i + E_j − I_j − 2 \cdot C_{ij}$ , onde $C_{ij}$ é o número de arestas entre $i$ e $j$.

Observe que $\Delta_{ij}$ é o valor da redução do número de arestas entre grupos ao se trocar os nós $i$ e $j$ de grupos.

O algoritmo de **Kernighan-Lin** assume que a rede é não direcionada, e é necessário determinar o número de vértices em cada grupo como entrada ao algoritmo ou grupos aproximadamente iguais são escolhidos aleatoriamente.

O algoritmo finaliza quando não existem nós $i$ e $j$ que possam ser trocados de grupo e diminuir o valor da função objetivo, ou após um número fixo de iterações pré-estabelecido.

### Exemplo 4
Aplicação do algoritmo **Kernighan-Lin** do **networkx** a uma rede de personagens do livro 2 da saga Harry Potter.

In [ ]:
# algoritmo Kernighan-Lin aplicado a uma rede
com1, com2 = cm.kernighan_lin_bisection(gpotter.to_undirected(), max_iter=100)

print(f"com1: {com1}")
print(f"com2: {com2}")

In [ ]:
print('comunidade 1:')
for k in com1:
    print(f"{k}: {gpotter.nodes[k]['nome']}")

In [ ]:
print('Comunidade 2:')
for k in com2:
    print(f"{k}: {gpotter.nodes[k]['nome']}")

In [ ]:
com1 = list(com1)
com2 = list(com2)

fig, ax = plt.subplots(1, 1, figsize=(15,10))

pos = nx.circular_layout(gpotter)

for k in range(len(com1)):
    pos[com1[k]][0] = 300*np.cos(2*np.pi*k/len(com1))
    pos[com1[k]][1] = 300*np.sin(2*np.pi*k/len(com1))+700

for k in range(len(com2)):
    pos[com2[k]][0] = 300*np.cos(2*np.pi*k/len(com2))
    pos[com2[k]][1] = 300*np.sin(2*np.pi*k/len(com2))    

cores = []
for no in gpotter.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(gpotter,pos=pos,edge_color='lightgray',
                 node_size = 500, labels=etiquetas,
                 font_size=10, node_color=cores)
plt.box(False)
plt.show()

### Exemplo 5
Cálculo de comunidades para o Karate Club Graph.

In [ ]:
KCG = nx.karate_club_graph()

In [ ]:
# algoritmo Kernighan-Lin aplicado a uma rede
com1, com2 = cm.kernighan_lin_bisection(KCG.to_undirected(), max_iter=100)

print(f"com1: {com1}")
print(f"com2: {com2}")

In [ ]:
fig, ax = plt.subplots(1, 1, figsize=(15,10))

pos = nx.kamada_kawai_layout(KCG)

cores = []
for no in KCG.nodes():
    if no in com1:
        cores.append('lightskyblue')
    else:
        cores.append('steelblue')

#etiquetas = nx.get_node_attributes(gpotter, 'nome')

nx.draw_networkx(KCG, pos=pos, edge_color='lightgray',
                 node_size=500, with_labels=True,
                 font_size=10, node_color=cores)
plt.box(False)
plt.show()